# EVPN with Nornir
* This lab is really about EVPN-MPLS and not EVPN-VXLAN

## VPLS Disadvantages (a.k.a. Why EVPN?)
* Mac addresses learned in the data plane (flood-and-learn)
    * MAC tables can become inconsistent (tables are distributed)
    * Difficult to apply policy
* Multihoming requires shutting down loops
    * Active/Active multihoming not possible
    * Requires STP (yuck) or other methods
* Only one **active** IRB at a time
* Multiple (3) signaling methods
    * Learning VPLS but * 3

    
## EVPN Overview
* RFC 7432 BGP MPLS-Based Ethernet VPN
* Fixes the disadvantages of VPLS
* Acts like a distributed switch
* Uses BGP for MAC information, IRB interfaces, and multihoming
    * All PEs share the same view of the MAC table
    * Moves MAC learning to the control plane
    * MAC-IP bindings learned
    * Snoops on ARP traffic
    * PEs can answer local ARP requests (ARP Suppression)
* Apply routing policies to MAC learning
* Mobility Sequence Count (counter for individual MAC moves)
    * Advertised via the extended community 'MAC Mobility community'
* Multihomed clients can be advertised in BGP and run in active/active
    * No STP, no shutting down links
    * 'Ethernet segment': set of Ethernet links that connect to the same CE device; **BUT** generally used to refer to one specific single-homed link in real life
    * ESI: Ethernet Segment Identifier, 10-octet, non-zero number
        * Multihomed CEs will have the same ESI on their associated links
        * All-zero indicates single-homed
* All IRBs can be active at the same time
    * No VRRP needed!
* Can use EVPN-MPLS (use LDP labels; what this lab is about) or EVPN-VXLAN (no LDP label needed; not in this lab)
* 8 route types (and growing); EVPN-MPLS uses types 1-4
    * Type 1: Ethernet Auto-Discovery Route
    * Type 2: MAC/IP Advertisement Route
    * Type 3: Inclusive Multicast Ethernet Tag Route
    * Type 4: Ethernet Segment Route
* AFI: Layer-2 VPN (25); SAFI: EVPN (70)

## EVPN Terms

* EVI - EVPN Instance (spanning all PEs according to RFC); sometimes used to refer to just local PE

* Type 2 Routes
    * Advertises MAC addresses OR MAC/IP address bindings
    * Route Target used to id the VPN to which it belongs
    * Includes Route-Distinguisher, VPN Label, and much more
    
* Type 3 Route: Inclusive Multicast Ethernet Tag Route
    * PMSI - Provider Multicast Service Interface Tunnel (The multicast tunnel)
    * Multicast VPN can use inclusive multicast trees (all PEs) or selective (none, some, or all)
        * Type 3 routes are for **inclusive trees**, as per the name
        * Associated with a Bridge-Domain, represented by the Ethernet Tag ID
    * Join a multicast tree via BGP update with the 'Provider Multicast Service Interface Tunnel' attribute
        * BGP update states Tunnel Type (Ingress replication vs Multicast replication), MPLS Label, Tunnel ID
        * MPLS VPN label used for flooded traffic
    * Can use 'Ingress Replication' rather than true multicast

## Configs

* VLAN-Based EVI
    * Use instance-type evpn
    * use vlan-id <#>
    * One bridge domain with automatic VLAN normalization
* VLAN-Aware EVI
    * Use instance-type virtual-switch
    * Each bridge-domain aware of the VLAN ID that it hosts

##### Basic VLAN-based EVPN
```
# Turn on EVPN address family
set protocols bgp group INTERNAL family evpn signaling

# Optional, used for L3VPNs, but can be combined with EVPN-MPLS:
set protocols bgp group INTERNAL family inet-vpn unicast

# For RR if OOB
set routing-options resolution rib bgp.evpn.0 resolution-ribs inet.0

# Suggested to enable load-balancing
set policy-options policy-statement LB then load-balance per-packet # newer Junos versions actuallly have the 'per-flow' statement
set routing-options forwarding-table export LB

# Routing Instance for VLAN-Based EVI
set routing-instances KICIAKOCIA instance-type evpn
set routing-instances KICIAKOCIA vlan-id 90      <---VLAN-based; VLANs automatically normalized 
set routing-instances KICIAKOCIA ge-0/0/1.90
set routing-instances KICIAKOCIA ge-0/0/2.90
set routing-instances KICIAKOCIA ge-0/0/3.90
set routing-instances KICIAKOCIA route-distinguisher 10.1.1.1:90 
set routing-instances KICIAKOCIA vrf-target target:64512:90
set routing-instances KICIAKOCIA protcols evpn
```

##### CE Facing Interface
```
set interfaces ge0/0/4 flexible-vlan-tagging
set interfaces ge0/0/4 encapsulation flexible-ethernet-services
set interfaces ge0/0/4 unit 90 encapsulation vlan-bridge
set interfaces ge0/0/4 unit 90 vlan-id 90
set interfaces ge0/0/4 unit 90 family bridge
```


## Topology
To be added

## 